In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras import Sequential
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn import set_config
from numpy.linalg import norm
import pickle
from tqdm import tqdm
#resnet model
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

: 

In [ ]:
# creating instance (object) of the model
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))

: 

In [ ]:
# disabling training the model 
model.trainable= False

: 

#### Why do we update the last layer of the model?
while finding the images that are closest to the one uploaded by the user, we need to calculate the eucledian distance of the vector created from the uploaded image with every single vector created from the images in the database. To reduce the complexity and hence the time required for the same, we need to reduce the dimensionality of the vector of features produced by the model. By default, the model produces a matrix of 4 dimensions, the GlobalMaxPooling2D layer reduces that to 2. The model scans the image, the CNN layers go through each individual pixel, try to analyse the patterns and ultimately produce a matrix of 2048 features which the model thinks best seperates the features apart from one another.

In [ ]:
# changing the top layer of the model and replacing it with GlobalMaxPooling2D layer
model = Sequential([
    model,  #uptil here the model is the same as that which we imported
    GlobalMaxPooling2D()   #the new GlobalMaxPooling2D layer
])

: 

In [ ]:
model.summary()

: 

As we can see, the output shape of the GlobalMaxPooling2D() layer is a 2D matrix instead of 4 which is what we wanted.

In [ ]:
img = image.load_img('images/1163.jpg',target_size=(224,224))

: 

In [ ]:
print(type(img))

: 

In [ ]:
img_array = image.img_to_array(img)

: 

In [ ]:
img_array.shape

: 

Since the keras Resnet50 model takes in a 4D matrix as an input instead of a 3D matrix, where the extra dimension refers to batch size; we have to convert this 3D matrix image into a 4D one. 

In [ ]:
# adding an extra dimension to the image array
reshaped_img = np.expand_dims(img_array,axis=0)

: 

In [ ]:
reshaped_img.shape

: 

In [ ]:
reshaped_img

: 

In [ ]:
# changing the array slightly to make sure it comes in the format of the input of the model.
preprocessed_img_arr = preprocess_input(reshaped_img)

: 

In [ ]:
preprocessed_img_arr

: 

This **preprocessed_img_arr** becomes the input for the model which can be used to predict the 2048 features and return a vector of the same

In [ ]:
def reshape_image_to_input(path):
    img = image.load_img(path,target_size=(224,224))
    img_array = image.img_to_array(img)
    reshaped_img = np.expand_dims(img_array,axis=0)
    preprocessed_img_arr = preprocess_input(reshaped_img)
    return preprocessed_img_arr

: 

In [ ]:
def flatten_image_array_to1D(array):
    array = array.flatten()
    return array

: 

In [ ]:
pr = reshape_image_to_input('images/1540.jpg')

: 

In [ ]:
pr.shape

: 

In [ ]:
prediction = model.predict(pr)

: 

In [ ]:
pd.DataFrame(prediction)

: 

In [ ]:
# scaler = ColumnTransformer([
#     ('scaler',MinMaxScaler(feature_range=(0.1, 0.9)),slice(0,2048))
# ])
def scaler_func(data):
    return data/norm(data)

: 

In [ ]:
flat_pred=flatten_image_array_to1D(prediction)
flat_pred

: 

In [ ]:
arr = scaler_func(prediction)

: 

In [ ]:
arr

: 

In [ ]:
flat_arr = flatten_image_array_to1D(arr)
flat_arr

: 

### Creating pipeline

In [ ]:
# first transformer
image_array_transformer = FunctionTransformer(func=reshape_image_to_input)

: 

In [ ]:
#second transformer
def pred_with_model(data):
    data_pred = model.predict(data)
    return data_pred

model_transformer = FunctionTransformer(func=pred_with_model)    

: 

In [ ]:
#third transformer
scaler_f = FunctionTransformer(func=scaler_func)

: 

In [ ]:
#fourth transformer 
flatten = FunctionTransformer(func=flatten_image_array_to1D)

: 

In [ ]:
# pipeline
model_pipeline = Pipeline([
    ('image_to_array',image_array_transformer),
    ('model',model_transformer),
    ('scaler',scaler_f),
    ('flatten',flatten)
])

: 

In [ ]:
predicted_arr = model_pipeline.fit_transform('images/1551.jpg')

: 

In [ ]:
predicted_arr

: 

In [ ]:
# exporting the pipeline
pickle.dump(model_pipeline,open('model.pkl','wb'))

: 

In [ ]:
set_config(display='diagram')

: 

In [ ]:
model2= pickle.load(open('model.pkl','rb'))

: 

In [ ]:
yaay = model2.fit_transform('images/1565.jpg')

: 

In [ ]:
yaay2 = model2.fit_transform('images/1566.jpg')

: 

In [ ]:
import os

: 

In [ ]:
filenames = []

for file in os.listdir('images'):
    filenames.append(os.path.join('images',file))

: 

In [ ]:
filenames[:5]

: 

In [ ]:
feature_list = []
for file in tqdm(filenames):
    feature_list.append(model2.fit_transform(file))

: 

: 

: 

: 

: 